In [ ]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow import nn
import collections
from rdkit.Chem import Descriptors
from rdkit import Chem
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, multilabel_confusion_matrix, mean_squared_error
import statsmodels.api as sm

In [ ]:
data = pd.read_csv('ratio_dataset.csv', sep="\t",low_memory=False)

In [ ]:
#params
#for quick prototyping epoch count is set to 20
epoch_count = 20

In [ ]:
#Generate morgan FP, may take a while
bins_len= 512
smiles_train = data["RP"]
def vectorize(smiles):
    x_one = np.zeros((len(data), 8, bins_len ),dtype=np.int8)
    #x_one = []
    for ca,smil in enumerate(smiles):
        #x_one_ = []
        qq = list(smil.split("."))
        for fa in range(0,8):
            try:
                x_one[ca][fa]=np.fromiter((map(int, AllChem.GetMorganFingerprintAsBitVect(AllChem.MolFromSmiles(qq[fa]),3,nBits=bins_len).ToBitString())),int)
            except:
                pass
            #x_one_.append([0 for x in range(0,bins_len)])
            #print(smil)
        
    return x_one
X_train = vectorize(smiles_train.values)

In [ ]:
#saving a numpy array is recommended, to avoid generating morgan FP every time
#np.save('X_train_rat.npy', X_train)
# npy file can be loaded with:
#X_train = np.load("X_train_rat.npy")

In [ ]:
# vectorize output
one_hot_yz = np.zeros((len(data), 1 ),dtype=np.float16)
for i,nam in enumerate(data['rat2'])
    one_hot_yz[i] = float(nam)          
Y_train = np.array(one_hot_yz)


In [ ]:
# split dataset
X_train = np.array(X_train)
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,Y_train, test_size=0.1,shuffle=True)

In [ ]:
inputA = keras.Input(shape=(8,512),name='inputx')


x = layers.Dense(64, activation="relu")(inputA)

x = layers.Dense(32, activation="relu")(x)

x = layers.Flatten()(x)

p = layers.Dense(32, activation='relu')(x)

p = layers.Dense(1, activation="linear",dtype='float32',name="ratio")(p)


modelx = keras.Model(inputs=inputA, outputs=p)

modelx.compile(loss='mse', optimizer="adam")
print (modelx.summary())

In [ ]:
history = modelx.fit(X_train2,y_train2_2, epochs=epoch_count, batch_size=256, validation_split = 0.1)

In [ ]:
y_pred2=modelx.predict(X_test2,verbose=1)

print('MSE: ',mean_squared_error([float(x) for x in y_test2_2], [float(x) for x in y_pred2]))

#plt.axis([0, 1.1, 0, 1.1])
#plt.scatter((y_pred2),(y_test2_2))
print('Correlation cof matrix:')
print(np.corrcoef([float(x) for x in y_pred2], [float(x) for x in y_test2_2]))
X_addC = sm.add_constant([float(x) for x in y_test2_2])
result = sm.OLS([float(x) for x in y_pred2], X_addC).fit()
print('R2:',result.rsquared,'R2 adj:', result.rsquared_adj)

In [ ]:
inputA = keras.Input(shape=(8,512),name='inputx')

x = layers.LSTM(32,return_sequences=False)(inputA)

p = layers.Dense(64, activation='relu')(x)

p = layers.Dense(32, activation='relu')(x)

p = layers.Dense(1, activation="linear",dtype='float32',name="ratio")(p)


modelx = keras.Model(inputs=inputA, outputs=p)

modelx.compile(loss='mse', optimizer="adam")
print (modelx.summary())

In [ ]:
history = modelx.fit(X_train2,y_train2_2, epochs=epoch_count, batch_size=256, validation_split = 0.1)

In [ ]:
y_pred2=modelx.predict(X_test2,verbose=1)

print('MSE: ',mean_squared_error([float(x) for x in y_test2_2], [float(x) for x in y_pred2]))

#plt.axis([0, 1.1, 0, 1.1])
#plt.scatter((y_pred2),(y_test2_2))
print('Correlation cof matrix:')
print(np.corrcoef([float(x) for x in y_pred2], [float(x) for x in y_test2_2]))
X_addC = sm.add_constant([float(x) for x in y_test2_2])
result = sm.OLS([float(x) for x in y_pred2], X_addC).fit()
print('R2:',result.rsquared,'R2 adj:', result.rsquared_adj)

In [ ]:
inputA = keras.Input(shape=(8,512),name='inputx')

x = layers.Permute((2,1))(inputA)

k = layers.Conv1D(32,12,activation='relu')(x)
k = layers.AveragePooling1D(3)(k)
k = layers.Conv1D(32,12,activation='relu')(k)
k = layers.AveragePooling1D(3)(k)
k = layers.Conv1D(32,5,activation='relu')(k)
k = layers.AveragePooling1D(2)(k)

k = layers.Flatten()(k)

p = layers.Dense(32, activation='relu')(k)

#p = layers.Dense(64, activation='relu')(p)

p = layers.Dense(1, activation="linear",dtype='float32',name="ratio")(p)


modelx = keras.Model(inputs=inputA, outputs=p)

modelx.compile(loss='mse', optimizer="adam")
modelx.summary()

In [ ]:
history = modelx.fit(X_train2,y_train2_2, epochs=epoch_count, batch_size=256, validation_split = 0.1)

In [ ]:
y_pred2=modelx.predict(X_test2,verbose=1)

print('MSE: ',mean_squared_error([float(x) for x in y_test2_2], [float(x) for x in y_pred2]))

#plt.axis([0, 1.1, 0, 1.1])
#plt.scatter((y_pred2),(y_test2_2))
print('Correlation cof matrix:')
print(np.corrcoef([float(x) for x in y_pred2], [float(x) for x in y_test2_2]))
X_addC = sm.add_constant([float(x) for x in y_test2_2])
result = sm.OLS([float(x) for x in y_pred2], X_addC).fit()
print('R2:',result.rsquared,'R2 adj:', result.rsquared_adj)